In [1]:
import pandas as pd
import os

In [2]:
os.chdir(r'C:\SML_Projects\SML_CVE_type_cwe_predict')

In [3]:
df = pd.read_csv("data/preprocessed/preprocessed_dataset.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102963 entries, 0 to 102962
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   cve_id                      102963 non-null  float64
 1   description                 102963 non-null  float64
 2   cvss_score                  102963 non-null  float64
 3   cwe                         102963 non-null  float64
 4   vendor                      102963 non-null  float64
 5   product                     102963 non-null  float64
 6   publish_date                102963 non-null  float64
 7   type                        102963 non-null  float64
 8   vendor_freq                 102963 non-null  float64
 9   product_freq                102963 non-null  float64
 10  desc_len                    102963 non-null  float64
 11  desc_word_count             102963 non-null  float64
 12  desc_num_count              102963 non-null  float64
 13  desc_upper_rat

In [5]:
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10296 entries, 0 to 10295
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cve_id                      10296 non-null  float64
 1   description                 10296 non-null  float64
 2   cvss_score                  10296 non-null  float64
 3   cwe                         10296 non-null  float64
 4   vendor                      10296 non-null  float64
 5   product                     10296 non-null  float64
 6   publish_date                10296 non-null  float64
 7   type                        10296 non-null  float64
 8   vendor_freq                 10296 non-null  float64
 9   product_freq                10296 non-null  float64
 10  desc_len                    10296 non-null  float64
 11  desc_word_count             10296 non-null  float64
 12  desc_num_count              10296 non-null  float64
 13  desc_upper_ratio            102

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsOneClassifier

In [8]:
rare_types = df['type'].value_counts()[df['type'].value_counts() < 3].index
rare_cvss_scores  = df['cvss_score'].value_counts()[df['cvss_score'].value_counts() < 3].index

df = df.query('type not in @rare_types and cvss_score not in @rare_cvss_scores')

In [9]:
x = df.drop(['type', 'cvss_score'], axis=1)   
y = df[['type', 'cvss_score']] 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
print(y['type'].value_counts())
print(y['cvss_score'].value_counts())

type
4.0     4332
10.0    1940
3.0      997
8.0      821
7.0      760
2.0      560
5.0      256
1.0      251
6.0      184
0.0      111
9.0       84
Name: count, dtype: int64
cvss_score
3.0    5471
1.0    3358
0.0    1030
2.0     437
Name: count, dtype: int64


In [11]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Logistic Regression

In [12]:
lr = OneVsOneClassifier(LogisticRegression())
multi_lr = MultiOutputClassifier(lr)

multi_lr.fit(x_train, y_train)
y_pred = multi_lr.predict(x_test)

lr_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
lr_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

lr_scores_type = cross_val_score(lr, x, y['type'], cv=kf, scoring='f1_macro')
lr_scores_cvss_score  = cross_val_score(lr, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Logistic Regression Accuracy for 'type':", lr_accuracy_type)
print("Logistic Regression Accuracy for 'cvss_score' :", lr_accuracy_cvss_score)

print("K-Fold mean F1 (type):", lr_scores_type.mean())
print("K-Fold std  F1 (type):", lr_scores_type.std())

print("K-Fold mean F1 (cvss_score):", lr_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", lr_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Logistic Regression Accuracy for 'type': 0.7781553398058253
Logistic Regression Accuracy for 'cvss_score' : 0.5927184466019417
K-Fold mean F1 (type): 0.6445131949596711
K-Fold std  F1 (type): 0.009579123963630845
K-Fold mean F1 (cvss_score): 0.30463156480706194
K-Fold std  F1 (cvss_score): 0.007143754486132826

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.95      0.78      0.86        50
         2.0       0.85      0.89      0.87       103
         3.0       0.23      0.13      0.17       200
         4.0       0.71      0.87      0.78       834
         5.0       0.93      0.63      0.75        41
         6.0       0.65      0.42      0.51        36
         7.0       0.81      0.68      0.74       155
         8.0       1.00      0.97      0.98       176
         9.0       1.00      0.50      0.67        14
        10.0       0.95      0.95      0.95       423

  

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Decision Tree

In [13]:
dt = OneVsOneClassifier(DecisionTreeClassifier(random_state=42))
multi_dt = MultiOutputClassifier(dt)

multi_dt.fit(x_train, y_train)
y_pred = multi_dt.predict(x_test)

dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
dt_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

dt_scores_type = cross_val_score(dt, x, y['type'], cv=kf, scoring='f1_macro')
dt_scores_cvss_score  = cross_val_score(dt, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Decision Tree Accuracy for 'type':", dt_accuracy_type)
print("Decision Tree Accuracy for 'cvss_score' :", dt_accuracy_cvss_score)

print("K-Fold mean F1 (type):", dt_scores_type.mean())
print("K-Fold std  F1 (type):", dt_scores_type.std())

print("K-Fold mean F1 (cvss_score):", dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", dt_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Decision Tree Accuracy for 'type': 0.9383495145631068
Decision Tree Accuracy for 'cvss_score' : 0.6116504854368932
K-Fold mean F1 (type): 0.9133093260202317
K-Fold std  F1 (type): 0.0018874920851546543
K-Fold mean F1 (cvss_score): 0.48894931768249084
K-Fold std  F1 (cvss_score): 0.01268348472828913

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.68      0.81        28
         1.0       0.98      0.94      0.96        50
         2.0       0.84      0.86      0.85       103
         3.0       0.96      0.92      0.94       200
         4.0       0.94      0.96      0.95       834
         5.0       0.95      0.88      0.91        41
         6.0       0.73      0.75      0.74        36
         7.0       0.85      0.86      0.86       155
         8.0       0.99      0.99      0.99       176
         9.0       0.93      1.00      0.97        14
        10.0       0.97      0.97      0.97       423

    accuracy  

# RandomForest

In [14]:
rf = OneVsOneClassifier(RandomForestClassifier(random_state=42))
multi_rf = MultiOutputClassifier(rf)

multi_rf.fit(x_train, y_train)
y_pred = multi_rf.predict(x_test)

rf_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
rf_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

rf_scores_type = cross_val_score(rf, x, y['type'], cv=kf, scoring='f1_macro')
rf_scores_cvss_score  = cross_val_score(rf, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Random Forest Accuracy for 'type':", rf_accuracy_type)
print("Random Forest Accuracy for 'cvss_score' :", rf_accuracy_cvss_score)

print("K-Fold mean F1 (type):", rf_scores_type.mean())
print("K-Fold std  F1 (type):", rf_scores_type.std())

print("K-Fold mean F1 (cvss_score):", rf_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", rf_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Random Forest Accuracy for 'type': 0.929126213592233
Random Forest Accuracy for 'cvss_score' : 0.6781553398058252
K-Fold mean F1 (type): 0.877941423067492
K-Fold std  F1 (type): 0.014125568121715665
K-Fold mean F1 (cvss_score): 0.5459108208096971
K-Fold std  F1 (cvss_score): 0.007282076959797635

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.39      0.56        28
         1.0       0.96      0.90      0.93        50
         2.0       0.83      0.91      0.87       103
         3.0       0.88      0.91      0.89       200
         4.0       0.94      0.96      0.95       834
         5.0       0.94      0.80      0.87        41
         6.0       0.71      0.67      0.69        36
         7.0       0.85      0.86      0.86       155
         8.0       0.99      0.97      0.98       176
         9.0       0.93      1.00      0.97        14
        10.0       0.98      0.97      0.97       423

    accuracy     

# Gradient Boosting

In [15]:
gb = OneVsOneClassifier(GradientBoostingClassifier(random_state=42))
multi_gb = MultiOutputClassifier(gb)

multi_gb.fit(x_train, y_train)
y_pred = multi_gb.predict(x_test)

gb_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
gb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

gb_scores_type = cross_val_score(gb, x, y['type'], cv=kf, scoring='f1_macro')
gb_scores_cvss_score  = cross_val_score(gb, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Gradient Boosting Accuracy for 'type':", gb_accuracy_type)
print("Gradient Boosting Accuracy for 'cvss_score' :", gb_accuracy_cvss_score)

print("K-Fold mean F1 (type):", gb_scores_type.mean())
print("K-Fold std  F1 (type):", gb_scores_type.std())

print("K-Fold mean F1 (cvss_score):", gb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", gb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Gradient Boosting Accuracy for 'type': 0.9466019417475728
Gradient Boosting Accuracy for 'cvss_score' : 0.6592233009708738
K-Fold mean F1 (type): 0.922665563212218
K-Fold std  F1 (type): 0.0007815018282928408
K-Fold mean F1 (cvss_score): 0.5096708027510762
K-Fold std  F1 (cvss_score): 0.013294848325598503

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.75      0.86        28
         1.0       0.98      0.98      0.98        50
         2.0       0.84      0.95      0.89       103
         3.0       0.97      0.91      0.94       200
         4.0       0.94      0.96      0.95       834
         5.0       0.95      0.88      0.91        41
         6.0       0.78      0.81      0.79        36
         7.0       0.89      0.86      0.88       155
         8.0       0.99      0.99      0.99       176
         9.0       0.93      1.00      0.97        14
        10.0       0.98      0.97      0.98       423

    acc

# Extra Tree

In [16]:
et = OneVsOneClassifier(ExtraTreesClassifier(random_state=42))
multi_et = MultiOutputClassifier(et)

multi_et.fit(x_train, y_train)
y_pred = multi_et.predict(x_test)

et_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
et_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

et_scores_type = cross_val_score(et, x, y['type'], cv=kf, scoring='f1_macro')
et_scores_cvss_score  = cross_val_score(et, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Extra Tree Accuracy for 'type':", et_accuracy_type)
print("Extra Tree Accuracy for 'cvss_score' :", et_accuracy_cvss_score)

print("K-Fold mean F1 (type):", et_scores_type.mean())
print("K-Fold std  F1 (type):", et_scores_type.std())

print("K-Fold mean F1 (cvss_score):", et_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", et_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Extra Tree Accuracy for 'type': 0.9063106796116505
Extra Tree Accuracy for 'cvss_score' : 0.6888349514563107
K-Fold mean F1 (type): 0.8436406264456792
K-Fold std  F1 (type): 0.010570962506415045
K-Fold mean F1 (cvss_score): 0.5575543423386926
K-Fold std  F1 (cvss_score): 0.01011772439266333

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.78      0.25      0.38        28
         1.0       1.00      0.88      0.94        50
         2.0       0.83      0.95      0.89       103
         3.0       0.78      0.83      0.81       200
         4.0       0.90      0.95      0.93       834
         5.0       0.93      0.68      0.79        41
         6.0       0.69      0.61      0.65        36
         7.0       0.91      0.80      0.85       155
         8.0       1.00      0.98      0.99       176
         9.0       0.78      1.00      0.88        14
        10.0       0.97      0.95      0.96       423

    accuracy          

# Hist Gradient Boosting

In [17]:
hgb = OneVsOneClassifier(HistGradientBoostingClassifier(max_iter=200, random_state=42))
multi_hgb = MultiOutputClassifier(hgb)

multi_hgb.fit(x_train, y_train)
y_pred = multi_hgb.predict(x_test)

hgb_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
hgb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

hgb_scores_type = cross_val_score(hgb, x, y['type'], cv=kf, scoring='f1_macro')
hgb_scores_cvss_score  = cross_val_score(hgb, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Hist Gradient Boosting Accuracy for 'type':", hgb_accuracy_type)
print("Hist Gradient Boosting Accuracy for 'cvss_score' :", hgb_accuracy_cvss_score)

print("\nK-Fold mean F1 (type):", hgb_scores_type.mean())
print("K-Fold std  F1 (type):", hgb_scores_type.std())

print("\nK-Fold mean F1 (cvss_score):", hgb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", hgb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Hist Gradient Boosting Accuracy for 'type': 0.9495145631067962
Hist Gradient Boosting Accuracy for 'cvss_score' : 0.6893203883495146

K-Fold mean F1 (type): 0.9322928277747264
K-Fold std  F1 (type): 0.003507237966798782

K-Fold mean F1 (cvss_score): 0.5600617111528351
K-Fold std  F1 (cvss_score): 0.015807442767219616

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.75      0.86        28
         1.0       1.00      0.94      0.97        50
         2.0       0.86      0.95      0.90       103
         3.0       0.96      0.91      0.93       200
         4.0       0.95      0.96      0.95       834
         5.0       0.95      0.88      0.91        41
         6.0       0.82      0.86      0.84        36
         7.0       0.90      0.90      0.90       155
         8.0       1.00      0.99      0.99       176
         9.0       1.00      1.00      1.00        14
        10.0       0.97      0.98      0.98       

# KNN

In [18]:
knn = OneVsOneClassifier(KNeighborsClassifier(n_neighbors=1))
multi_knn = MultiOutputClassifier(knn)

multi_knn.fit(x_train, y_train)
y_pred = multi_knn.predict(x_test)

knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
knn_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

knn_scores_type = cross_val_score(knn, x, y['type'], cv=kf, scoring='f1_macro')
knn_scores_cvss_score  = cross_val_score(knn, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("KNN Accuracy for 'type':", knn_accuracy_type)
print("KNN Accuracy for 'cvss_score' :", knn_accuracy_cvss_score)

print("K-Fold mean F1 (type):", knn_scores_type.mean())
print("K-Fold std  F1 (type):", knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

KNN Accuracy for 'type': 0.8121359223300971
KNN Accuracy for 'cvss_score' : 0.6029126213592233
K-Fold mean F1 (type): 0.703665024476167
K-Fold std  F1 (type): 0.009585380349919074
K-Fold mean F1 (cvss_score): 0.47555034828233395
K-Fold std  F1 (cvss_score): 0.001946083182048487

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.43      0.21      0.29        28
         1.0       0.91      0.84      0.88        50
         2.0       0.78      0.78      0.78       103
         3.0       0.57      0.66      0.61       200
         4.0       0.81      0.85      0.83       834
         5.0       0.67      0.63      0.65        41
         6.0       0.58      0.31      0.40        36
         7.0       0.77      0.72      0.74       155
         8.0       0.97      0.97      0.97       176
         9.0       0.92      0.79      0.85        14
        10.0       0.92      0.90      0.91       423

    accuracy                       

# Adaboost

In [19]:
ab = OneVsOneClassifier(AdaBoostClassifier(n_estimators=200))
multi_ab = MultiOutputClassifier(ab)

multi_ab.fit(x_train, y_train)
y_pred = multi_ab.predict(x_test)

ab_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
ab_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

ab_scores_type = cross_val_score(ab, x, y['type'], cv=kf, scoring='f1_macro')
ab_scores_cvss_score  = cross_val_score(ab, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Adaboost Accuracy for 'type':", ab_accuracy_type)
print("Adaboost Accuracy for 'cvss_score' :", ab_accuracy_cvss_score)

print("K-Fold mean F1 (type):", ab_scores_type.mean())
print("K-Fold std  F1 (type):", ab_scores_type.std())

print("K-Fold mean F1 (cvss_score):", ab_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", ab_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Adaboost Accuracy for 'type': 0.9495145631067962
Adaboost Accuracy for 'cvss_score' : 0.5966019417475729
K-Fold mean F1 (type): 0.9265842110667909
K-Fold std  F1 (type): 0.0072568413087262746
K-Fold mean F1 (cvss_score): 0.37440394246918673
K-Fold std  F1 (cvss_score): 0.02393854036169785

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.96      0.89      0.93        28
         1.0       0.98      1.00      0.99        50
         2.0       0.85      0.96      0.90       103
         3.0       0.97      0.93      0.95       200
         4.0       0.96      0.94      0.95       834
         5.0       0.92      0.88      0.90        41
         6.0       0.79      0.92      0.85        36
         7.0       0.88      0.91      0.90       155
         8.0       1.00      0.99      0.99       176
         9.0       0.93      1.00      0.97        14
        10.0       0.96      0.98      0.97       423

    accuracy            

# LightGBM

In [20]:
from lightgbm import LGBMClassifier
lgbm = OneVsOneClassifier(LGBMClassifier(random_state=42))
multi_lgbm = MultiOutputClassifier(lgbm)

multi_lgbm.fit(x_train, y_train)
y_pred = multi_lgbm.predict(x_test)

lgbm_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
lgbm_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

lgbm_scores_type = cross_val_score(lgbm, x, y['type'], cv=kf, scoring='f1_macro')
lgbm_scores_cvss_score  = cross_val_score(lgbm, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("LightGBM Accuracy for 'type':", lgbm_accuracy_type)
print("LightGBM Accuracy for 'cvss_score' :", lgbm_accuracy_cvss_score)

print("K-Fold mean F1 (type):", lgbm_scores_type.mean())
print("K-Fold std  F1 (type):", lgbm_scores_type.std())

print("K-Fold mean F1 (cvss_score):", lgbm_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", lgbm_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

[LightGBM] [Info] Number of positive: 201, number of negative: 83
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 284, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.707746 -> initscore=0.884464
[LightGBM] [Info] Start training from score 0.884464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-package

[LightGBM] [Info] Number of positive: 169, number of negative: 71
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 240, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.704167 -> initscore=0.867219
[LightGBM] [Info] Start training from score 0.867219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-package

[LightGBM] [Info] Number of positive: 168, number of negative: 76
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 759
[LightGBM] [Info] Number of data points in the train set: 244, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.688525 -> initscore=0.793231
[LightGBM] [Info] Start training from score 0.793231
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

...

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-package

[LightGBM] [Info] Number of positive: 281, number of negative: 2246
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2686
[LightGBM] [Info] Number of data points in the train set: 2527, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.111199 -> initscore=-2.078551
[LightGBM] [Info] Start training from score -2.078551
[LightGBM] [Info] Number of positive: 3628, number of negative: 2246
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000641 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2934
[LightGBM] [Info] Number of data points in the train set: 5874, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.617637 -> initscore=0.479531
[LightGBM] [Info] Start training from score 0.479531
[LightGBM] [Info] Num

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-package

[LightGBM] [Info] Number of positive: 294, number of negative: 696
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2472
[LightGBM] [Info] Number of data points in the train set: 990, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.296970 -> initscore=-0.861770
[LightGBM] [Info] Start training from score -0.861770
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 3636, number of negative: 696
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000427 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2871
[LightGBM] [Info] Number of data points in the train set: 4332, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.839335 -> initscore=1.653290
[L

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-package

# Bagging

In [21]:
from sklearn.ensemble import BaggingClassifier

bagging = OneVsOneClassifier(BaggingClassifier(random_state=42))
multi_bag = MultiOutputClassifier(bagging)

multi_bag.fit(x_train, y_train)
y_pred = multi_bag.predict(x_test)

bag_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
bag_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

bag_scores_type = cross_val_score(bagging, x, y['type'], cv=kf, scoring='f1_macro')
bag_scores_cvss_score  = cross_val_score(bagging, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging Accuracy for 'type':", bag_accuracy_type)
print("Bagging Accuracy for 'cvss_score' :", bag_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_scores_type.mean())
print("K-Fold std  F1 (type):", bag_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging Accuracy for 'type': 0.9407766990291262
Bagging Accuracy for 'cvss_score' : 0.6587378640776699
K-Fold mean F1 (type): 0.9196217911235217
K-Fold std  F1 (type): 0.0022421192725903772
K-Fold mean F1 (cvss_score): 0.5300889965716599
K-Fold std  F1 (cvss_score): 0.014764951478196259

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.75      0.86        28
         1.0       0.96      0.98      0.97        50
         2.0       0.81      0.92      0.86       103
         3.0       0.95      0.91      0.93       200
         4.0       0.96      0.95      0.95       834
         5.0       0.95      0.88      0.91        41
         6.0       0.71      0.83      0.77        36
         7.0       0.86      0.88      0.87       155
         8.0       0.99      0.98      0.99       176
         9.0       0.93      1.00      0.97        14
        10.0       0.97      0.98      0.97       423

    accuracy              

# SVC

In [22]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', C=5, decision_function_shape='ovo')
multi_svc = MultiOutputClassifier(svc)

multi_svc.fit(x_train, y_train)
y_pred = multi_svc.predict(x_test)

svc_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
svc_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:,1])

svc_scores_type = cross_val_score(svc, x, y['type'], cv=kf, scoring='f1_macro')
svc_scores_cvss_score  = cross_val_score(svc, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("SVC Accuracy for 'type':", svc_accuracy_type)
print("SVC Accuracy for 'cvss_score' :", svc_accuracy_cvss_score)

print("K-Fold mean F1 (type):", svc_scores_type.mean())
print("K-Fold std  F1 (type):", svc_scores_type.std())

print("K-Fold mean F1 (cvss_score):", svc_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", svc_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

SVC Accuracy for 'type': 0.8495145631067961
SVC Accuracy for 'cvss_score' : 0.6330097087378641
K-Fold mean F1 (type): 0.7522140407362387
K-Fold std  F1 (type): 0.005939275133587859
K-Fold mean F1 (cvss_score): 0.35284066989479984
K-Fold std  F1 (cvss_score): 0.0032606218597976996

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.67      0.07      0.13        28
         1.0       0.95      0.78      0.86        50
         2.0       0.77      0.95      0.85       103
         3.0       0.62      0.62      0.62       200
         4.0       0.84      0.88      0.86       834
         5.0       0.93      0.68      0.79        41
         6.0       0.58      0.53      0.55        36
         7.0       0.86      0.77      0.82       155
         8.0       1.00      0.96      0.98       176
         9.0       0.85      0.79      0.81        14
        10.0       0.95      0.95      0.95       423

    accuracy                     

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Hard Voting

In [24]:
from sklearn.ensemble import VotingClassifier

model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_hard = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='hard'
)

multi_voting_hard = MultiOutputClassifier(voting_hard)

multi_voting_hard.fit(x_train, y_train)
y_pred = multi_voting_hard.predict(x_test)

hard_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
hard_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])


hard_scores_type = cross_val_score(voting_hard, x, y['type'], cv=kf, scoring='f1_macro')
hard_scores_cvss_score  = cross_val_score(voting_hard, x, y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Hard Voting Accuracy for type:", hard_acc_type)
print("Hard Voting Accuracy for cvss_score :", hard_acc_cvss_score)

print("K-fold F1 mean (type):", hard_scores_type.mean())
print("K-fold F1 std  (type):", hard_scores_type.std())

print("K-fold F1 mean (cvss_score) :", hard_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", hard_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Hard Voting Accuracy for type: 0.9092233009708738
Hard Voting Accuracy for cvss_score : 0.6849514563106797
K-fold F1 mean (type): 0.8535412937991548
K-fold F1 std  (type): 0.00542323256362824
K-fold F1 mean (cvss_score) : 0.546800250140797
K-fold F1 std  (cvss_score) : 0.004329239698999266
              precision    recall  f1-score   support

         0.0       0.89      0.29      0.43        28
         1.0       0.98      0.90      0.94        50
         2.0       0.82      0.95      0.88       103
         3.0       0.82      0.83      0.83       200
         4.0       0.90      0.95      0.93       834
         5.0       0.90      0.68      0.78        41
         6.0       0.68      0.53      0.59        36
         7.0       0.90      0.81      0.85       155
         8.0       0.99      0.98      0.99       176
         9.0       1.00      1.00      1.00        14
        10.0       0.97      0.95      0.96       423

    accuracy                           0.91      2060
   ma

# Soft Voting

In [25]:
model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_hard = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='soft'
)

multi_voting_hard = MultiOutputClassifier(voting_hard)

multi_voting_hard.fit(x_train, y_train)
y_pred = multi_voting_hard.predict(x_test)

soft_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
soft_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

soft_scores_type = cross_val_score(voting_hard, x, y['type'], cv=kf, scoring='f1_macro')
soft_scores_cvss_score  = cross_val_score(voting_hard, x, y['cvss_score'],  cv=kf, scoring='f1_macro')


print("Hard Voting Accuracy for type:", soft_acc_type)
print("Hard Voting Accuracy for cvss_score :", soft_acc_cvss_score)

print("K-fold F1 mean (type):", soft_scores_type.mean())
print("K-fold F1 std  (type):", soft_scores_type.std())

print("K-fold F1 mean (cvss_score) :", soft_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", soft_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Hard Voting Accuracy for type: 0.9063106796116505
Hard Voting Accuracy for cvss_score : 0.6786407766990291
K-fold F1 mean (type): 0.8401265096947981
K-fold F1 std  (type): 0.004807786214563205
K-fold F1 mean (cvss_score) : 0.53279587273932
K-fold F1 std  (cvss_score) : 0.00474278913678708
              precision    recall  f1-score   support

         0.0       1.00      0.25      0.40        28
         1.0       0.98      0.88      0.93        50
         2.0       0.83      0.97      0.89       103
         3.0       0.82      0.83      0.83       200
         4.0       0.89      0.95      0.92       834
         5.0       0.90      0.68      0.78        41
         6.0       0.68      0.53      0.59        36
         7.0       0.89      0.79      0.84       155
         8.0       1.00      0.98      0.99       176
         9.0       1.00      0.86      0.92        14
        10.0       0.97      0.95      0.96       423

    accuracy                           0.91      2060
   mac

# Stacking

In [26]:
from sklearn.ensemble import StackingClassifier

base1 = RandomForestClassifier(n_estimators=200, random_state=42)
base2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
base3 = LogisticRegression(max_iter=500)

stacking = StackingClassifier(
    estimators=[
        ('rf', base1),
        ('et', base2),
        ('lr', base3)
    ],
    final_estimator=LogisticRegression(max_iter=500)
)

multi_stacking = MultiOutputClassifier(stacking)

multi_stacking.fit(x_train, y_train)
y_pred = multi_stacking.predict(x_test)

stacking_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
stacking_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

stacking_scores_type = cross_val_score(stacking, x, y['type'], cv=kf, scoring='f1_macro')
stacking_scores_cvss_score  = cross_val_score(stacking, x, y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Stacking Accuracy for type:", stacking_acc_type)
print("Stacking Accuracy for cvss_score :", stacking_acc_cvss_score)

print("K-fold F1 mean (type):", stacking_scores_type.mean())
print("K-fold F1 std  (type):", stacking_scores_type.std())

print("K-fold F1 mean (cvss_score) :", stacking_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", stacking_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Stacking Accuracy for type: 0.9276699029126214
Stacking Accuracy for cvss_score : 0.6936893203883495
K-fold F1 mean (type): 0.8767479622209979
K-fold F1 std  (type): 0.008390731109801819
K-fold F1 mean (cvss_score) : 0.5663912915765845
K-fold F1 std  (cvss_score) : 0.017561396746994373
              precision    recall  f1-score   support

         0.0       0.80      0.43      0.56        28
         1.0       1.00      0.92      0.96        50
         2.0       0.85      0.93      0.89       103
         3.0       0.83      0.90      0.86       200
         4.0       0.94      0.96      0.95       834
         5.0       0.88      0.71      0.78        41
         6.0       0.72      0.58      0.65        36
         7.0       0.90      0.83      0.86       155
         8.0       1.00      0.98      0.99       176
         9.0       0.88      1.00      0.93        14
        10.0       0.97      0.97      0.97       423

    accuracy                           0.93      2060
   macro 

# Bagged KNN

In [27]:
bag_knn = BaggingClassifier(
    estimator=OneVsOneClassifier(KNeighborsClassifier(n_neighbors=1)),
    n_estimators=100,
    random_state=42
)

multi_bag_knn = MultiOutputClassifier(bag_knn)

multi_bag_knn.fit(x_train, y_train)
y_pred = multi_bag_knn.predict(x_test)

bag_knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_knn_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_knn_scores_type = cross_val_score(bag_knn, x_train, y_train['type'], cv=kf, scoring='f1_macro')
bag_knn_scores_cvss_score = cross_val_score(bag_knn, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging OVO KNN Accuracy for 'type':", bag_knn_accuracy_type)
print("Bagging OVO KNN Accuracy for 'cvss_score':", bag_knn_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_knn_scores_type.mean())
print("K-Fold std  F1 (type):", bag_knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging OVO KNN Accuracy for 'type': 0.8121359223300971
Bagging OVO KNN Accuracy for 'cvss_score': 0.6033980582524272
K-Fold mean F1 (type): 0.6873676833719826
K-Fold std  F1 (type): 0.008717440573823714
K-Fold mean F1 (cvss_score): 0.4635969724423244
K-Fold std  F1 (cvss_score): 0.005687439449563473

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.43      0.21      0.29        28
         1.0       0.91      0.84      0.88        50
         2.0       0.78      0.78      0.78       103
         3.0       0.57      0.66      0.61       200
         4.0       0.81      0.85      0.83       834
         5.0       0.67      0.63      0.65        41
         6.0       0.58      0.31      0.40        36
         7.0       0.77      0.72      0.74       155
         8.0       0.97      0.97      0.97       176
         9.0       0.92      0.79      0.85        14
        10.0       0.92      0.90      0.91       423

    accuracy

# Bagged DecisionTree

In [28]:
bag_dt = BaggingClassifier(
    estimator=OneVsOneClassifier(DecisionTreeClassifier()),
    n_estimators=100,
    random_state=42
)

multi_bag_dt = MultiOutputClassifier(bag_dt)

multi_bag_dt.fit(x_train, y_train)
y_pred = multi_bag_dt.predict(x_test)

bag_dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_dt_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_dt_scores_type = cross_val_score(bag_dt, x_train, y_train['type'], cv=kf, scoring='f1_macro')
bag_dt_scores_cvss_score = cross_val_score(bag_dt, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging OVO DecisionTree Accuracy for 'type':", bag_dt_accuracy_type)
print("Bagging OVO DecisionTree Accuracy for 'cvss_score':", bag_dt_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_dt_scores_type.mean())
print("K-Fold std  F1 (type):", bag_dt_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_dt_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging OVO DecisionTree Accuracy for 'type': 0.945631067961165
Bagging OVO DecisionTree Accuracy for 'cvss_score': 0.6762135922330097
K-Fold mean F1 (type): 0.9272364810310253
K-Fold std  F1 (type): 0.008745884991283336
K-Fold mean F1 (cvss_score): 0.5120009531888522
K-Fold std  F1 (cvss_score): 0.018271439697289112

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.71      0.83        28
         1.0       0.98      0.94      0.96        50
         2.0       0.84      0.94      0.89       103
         3.0       0.96      0.90      0.93       200
         4.0       0.96      0.96      0.96       834
         5.0       0.95      0.88      0.91        41
         6.0       0.74      0.81      0.77        36
         7.0       0.86      0.91      0.88       155
         8.0       0.99      0.98      0.99       176
         9.0       0.82      1.00      0.90        14
        10.0       0.97      0.98      0.98       

In [29]:
from rich.table import Table
from rich.console import Console

console = Console()

results = [
    ['LogisticRegression', lr_accuracy_type, lr_scores_type.mean(), lr_scores_type.std(), lr_accuracy_cvss_score, lr_scores_cvss_score.mean(), lr_scores_cvss_score.std()],
    ['DecisionTree', dt_accuracy_type, dt_scores_type.mean(), dt_scores_type.std(), dt_accuracy_cvss_score, dt_scores_cvss_score.mean(), dt_scores_cvss_score.std()],
    ['RandomForest', rf_accuracy_type, rf_scores_type.mean(), rf_scores_type.std(), rf_accuracy_cvss_score, rf_scores_cvss_score.mean(), rf_scores_cvss_score.std()],
    ['ExtraTrees', et_accuracy_type, et_scores_type.mean(), et_scores_type.std(), et_accuracy_cvss_score, et_scores_cvss_score.mean(), et_scores_cvss_score.std()],
    ['GradientBoosting', gb_accuracy_type, gb_scores_type.mean(), gb_scores_type.std(), gb_accuracy_cvss_score, gb_scores_cvss_score.mean(), gb_scores_cvss_score.std()],
    ['HistGradientBoosting', hgb_accuracy_type, hgb_scores_type.mean(), hgb_scores_type.std(), hgb_accuracy_cvss_score, hgb_scores_cvss_score.mean(), hgb_scores_cvss_score.std()],
    ['KNN', knn_accuracy_type, knn_scores_type.mean(), knn_scores_type.std(), knn_accuracy_cvss_score, knn_scores_cvss_score.mean(), knn_scores_cvss_score.std()],
    ['AdaBoost', ab_accuracy_type, ab_scores_type.mean(), ab_scores_type.std(), ab_accuracy_cvss_score, ab_scores_cvss_score.mean(), ab_scores_cvss_score.std()],
    ['LightGBM', lgbm_accuracy_type, lgbm_scores_type.mean(), lgbm_scores_type.std(), lgbm_accuracy_cvss_score, lgbm_scores_cvss_score.mean(), lgbm_scores_cvss_score.std()],
    ['Bagging', bag_accuracy_type, bag_scores_type.mean(), bag_scores_type.std(), bag_accuracy_cvss_score, bag_scores_cvss_score.mean(), bag_scores_cvss_score.std()],
    ['Hard Voting', hard_acc_type, hard_scores_type.mean(), hard_scores_type.std(), hard_acc_cvss_score, hard_scores_cvss_score.mean(), hard_scores_cvss_score.std()],
    ['Soft Voting', soft_acc_type, soft_scores_type.mean(), soft_scores_type.std(), soft_acc_cvss_score, soft_scores_cvss_score.mean(), soft_scores_cvss_score.std()],
    ['Stacking', stacking_acc_type, stacking_scores_type.mean(), stacking_scores_type.std(), stacking_acc_cvss_score, stacking_scores_cvss_score.mean(), stacking_scores_cvss_score.std()],
    ['SVM', svc_accuracy_type, svc_scores_type.mean(), svc_scores_type.std(), svc_accuracy_cvss_score, svc_scores_cvss_score.mean(), svc_scores_cvss_score.std()],
    ['Bagged KNN', bag_knn_accuracy_type, bag_knn_scores_type.mean(), bag_knn_scores_type.std(), bag_knn_accuracy_cvss_score, bag_knn_scores_cvss_score.mean(), bag_knn_scores_cvss_score.std()],
    ['Bagged DT', bag_dt_accuracy_type, bag_dt_scores_type.mean(), bag_dt_scores_type.std(), bag_dt_accuracy_cvss_score, bag_dt_scores_cvss_score.mean(), bag_dt_scores_cvss_score.std()],
]

for row in results:
    type_acc = row[1]
    cvss_score_acc = row[4]
    combined = (type_acc + cvss_score_acc) / 2
    row.append(combined)

result_sorted = sorted(results, key=lambda i: i[-1], reverse=True)

best_model = max(results, key=lambda x: x[-1])
worst_model = min(results, key=lambda x: x[-1])

table = Table(title="OVO Comparison", show_lines=True)
table.add_column("Algorithm")
table.add_column("Type Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("cvss_score Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("Combined", justify="right")

for row in result_sorted:
    algo, type_acc, kmean_type, kstd_type, cvss_score_acc, kmean_cvss_score, kstd_cvss_score, combined = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{type_acc:.2f}[/bold green]",
            f"[bold green]{kmean_type:.2f}[/bold green]",
            f"[bold green]{kstd_type:.2f}[/bold green]",
            f"[bold green]{cvss_score_acc:.2f}[/bold green]",
            f"[bold green]{kmean_cvss_score:.2f}[/bold green]",
            f"[bold green]{kstd_cvss_score:.2f}[/bold green]",
            f"[bold green]{combined:.2f}[/bold green]",
        )
    elif row == worst_model:
        table.add_row(
            f"[bold red]{algo}[/bold red]",
            f"[bold red]{type_acc:.2f}[/bold red]",
            f"[bold red]{kmean_type:.2f}[/bold red]",
            f"[bold red]{kstd_type:.2f}[/bold red]",
            f"[bold red]{cvss_score_acc:.2f}[/bold red]",
            f"[bold red]{kmean_cvss_score:.2f}[/bold red]",
            f"[bold red]{kstd_cvss_score:.2f}[/bold red]",
            f"[bold red]{combined:.2f}[/bold red]",
        )
    else:
        table.add_row(
            algo, f"{type_acc:.2f}", f"{kmean_type:.2f}", f"{kstd_type:.2f}",
            f"{cvss_score_acc:.2f}", f"{kmean_cvss_score:.2f}", f"{kstd_cvss_score:.2f}", f"{combined:.2f}"
        )

console.print(table)

                                                  OVO Comparison                                                   
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ HistGradientBoost… │ 0.95     │ 0.93        │ 0.00       │ 0.69           │ 0.56        │ 0.02       │     0.82 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LightGBM           │ 0.94     │ 0.93        │ 0.00       │ 0.68           │ 0.56        │ 0.01       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.95     │ 0.93        │ 0.01       │ 0.68           │ 0.51        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.93     │ 0.88        │ 0.01       │ 0.69           │ 0.57        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.93     │ 0.88        │ 0.01       │ 0.68           │ 0.55        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.95     │ 0.92        │ 0.00       │ 0.66           │ 0.51        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagging            │ 0.94     │ 0.92        │ 0.00       │ 0.66           │ 0.53        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.91     │ 0.84        │ 0.01       │ 0.69           │ 0.56        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.91     │ 0.85        │ 0.01       │ 0.68           │ 0.55        │ 0.00       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.91     │ 0.84        │ 0.00       │ 0.68           │ 0.53        │ 0.00       │     0.79 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.94     │ 0.91        │ 0.00       │ 0.61           │ 0.49        │ 0.01       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.95     │ 0.93        │ 0.01       │ 0.60           │ 0.37        │ 0.02       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ SVM                │ 0.85     │ 0.75        │ 0.01       │ 0.63           │ 0.35        │ 0.00       │     0.74 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.81     │ 0.69        │ 0.01       │ 0.60           │ 0.46        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.81     │ 0.70        │ 0.01       │ 0.60           │ 0.48        │ 0.00       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.78     │ 0.64        │ 0.01    

In [30]:
os.makedirs('results', exist_ok=True)

temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/OVO_OVR_compare.txt', 'a', encoding='utf-8') as f:
    f.write(text)

                                                  OVO Comparison                                                   
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ HistGradientBoost… │ 0.95     │ 0.93        │ 0.00       │ 0.69           │ 0.56        │ 0.02       │     0.82 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LightGBM           │ 0.94     │ 0.93        │ 0.00       │ 0.68           │ 0.56        │ 0.01       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.95     │ 0.93        │ 0.01       │ 0.68           │ 0.51        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.93     │ 0.88        │ 0.01       │ 0.69           │ 0.57        │ 0.02       │     0.81 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.93     │ 0.88        │ 0.01       │ 0.68           │ 0.55        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.95     │ 0.92        │ 0.00       │ 0.66           │ 0.51        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagging            │ 0.94     │ 0.92        │ 0.00       │ 0.66           │ 0.53        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.91     │ 0.84        │ 0.01       │ 0.69           │ 0.56        │ 0.01       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.91     │ 0.85        │ 0.01       │ 0.68           │ 0.55        │ 0.00       │     0.80 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.91     │ 0.84        │ 0.00       │ 0.68           │ 0.53        │ 0.00       │     0.79 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.94     │ 0.91        │ 0.00       │ 0.61           │ 0.49        │ 0.01       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.95     │ 0.93        │ 0.01       │ 0.60           │ 0.37        │ 0.02       │     0.77 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ SVM                │ 0.85     │ 0.75        │ 0.01       │ 0.63           │ 0.35        │ 0.00       │     0.74 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.81     │ 0.69        │ 0.01       │ 0.60           │ 0.46        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.81     │ 0.70        │ 0.01       │ 0.60           │ 0.48        │ 0.00       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.78     │ 0.64        │ 0.01    